In [345]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import getpass  # To get the password without showing the input
import re
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import

In [249]:
password = getpass.getpass()

········


- Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features. Use the data from 2005.

- Create a query to get the list of films and a boolean indicating if it was rented last month (August 2005). This would be our target variable.

- Read the data into a Pandas dataframe.

- Analyze extracted features and transform them. You may need to encode some categorical variables, or scale numerical variables.

- Create a logistic regression model to predict this variable from the cleaned data.

- Evaluate the results.

In [250]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

In [261]:
film ='''
SELECT 
    t1.customer_id
    ,t1.inventory_id
    ,t2.film_id
    ,t3.title
    ,t6.name AS genre
    ,t3.length
    ,t4.name AS language
    ,t3.special_features
    ,t3.rating
    ,t2.store_id
    ,t1.staff_id
    ,t3.rental_duration
    ,t3.rental_rate
    ,t3.replacement_cost
    , CASE WHEN t1.rental_date LIKE '2005-05%%' THEN 'True'
        ELSE 'False'
        END AS 'test'
FROM 
    sakila.rental AS t1
INNER JOIN sakila.inventory AS t2 ON t1.inventory_id = t2.inventory_id
RIGHT JOIN sakila.film AS t3 ON t2.film_id = t3.film_id
INNER JOIN sakila.language AS t4 ON t3.language_id = t4.language_id
INNER JOIN sakila.film_category AS t5 ON t3.film_id = t5.film_id
INNER JOIN sakila.category AS t6 ON t5.category_id = t6.category_id
WHERE
    t1.rental_date LIKE '2005%%'
;

'''
dfr = pd.read_sql_query(film, engine)

In [319]:
film =  '''
SELECT f.film_id, COUNT(r.rental_id) rented_times,f.rental_duration, c.name as category, f.length, f.rating, f.special_features, 
CASE
WHEN r.rental_date BETWEEN '2005-05-01' AND '2005-05-31' THEN True
    ELSE False END AS test
FROM sakila.film f
Left JOIN inventory i
	ON f.film_id = i.film_id
JOIN sakila.rental r
	ON i.inventory_id = r.inventory_id
Join sakila.film_category fc
	On fc.film_id = f.film_id
Join sakila.category c
	On c.category_id = fc.category_id
Where r.rental_date Between '2005-01-01' AND '2005-12-31'
GROUP BY film_id, rental_duration,category, f.length, f.rating, test, f.special_features; 

'''


dfr = pd.read_sql_query(film, engine)

In [320]:
dfr.duplicated().sum()

0

In [321]:
dfr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1585 entries, 0 to 1584
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   film_id           1585 non-null   int64 
 1   rented_times      1585 non-null   int64 
 2   rental_duration   1585 non-null   int64 
 3   category          1585 non-null   object
 4   length            1585 non-null   int64 
 5   rating            1585 non-null   object
 6   special_features  1585 non-null   object
 7   test              1585 non-null   int64 
dtypes: int64(5), object(3)
memory usage: 99.2+ KB


In [264]:
dfr = dfr[['customer_id','inventory_id','film_id','title','genre','length','language','special_features','rating','store_id','staff_id','rental_duration','rental_rate','replacement_cost','test']]


In [265]:
dfr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15862 entries, 0 to 15861
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer_id       15862 non-null  int64  
 1   inventory_id      15862 non-null  int64  
 2   film_id           15862 non-null  int64  
 3   title             15862 non-null  object 
 4   genre             15862 non-null  object 
 5   length            15862 non-null  int64  
 6   language          15862 non-null  object 
 7   special_features  15862 non-null  object 
 8   rating            15862 non-null  object 
 9   store_id          15862 non-null  int64  
 10  staff_id          15862 non-null  int64  
 11  rental_duration   15862 non-null  int64  
 12  rental_rate       15862 non-null  float64
 13  replacement_cost  15862 non-null  float64
 14  test              15862 non-null  object 
dtypes: float64(2), int64(7), object(6)
memory usage: 1.8+ MB


In [266]:
#dfr = dfr.dropna()

In [267]:
dfr['test'].unique()

array(['True', 'False'], dtype=object)

In [268]:
dfr['special_features'].value_counts()

Trailers,Commentaries,Behind the Scenes                   1308
Trailers                                                  1139
Trailers,Commentaries                                     1129
Trailers,Behind the Scenes                                1122
Behind the Scenes                                         1108
Commentaries,Deleted Scenes,Behind the Scenes             1101
Commentaries                                              1089
Commentaries,Behind the Scenes                            1078
Trailers,Deleted Scenes                                   1047
Deleted Scenes,Behind the Scenes                          1036
Deleted Scenes                                            1023
Commentaries,Deleted Scenes                               1011
Trailers,Commentaries,Deleted Scenes,Behind the Scenes     983
Trailers,Commentaries,Deleted Scenes                       916
Trailers,Deleted Scenes,Behind the Scenes                  772
Name: special_features, dtype: int64

In [269]:
dfr.duplicated().sum()

0

In [322]:

la = []
lb = []
lc = []
ld = []

for i in dfr['special_features']:
        if re.findall('Trailers', i):
            la.append(1)
        else:
            la.append(0)
dfr['trailers'] = la

for i in dfr['special_features']:
        if re.findall('Behind the Scenes', i):
            lb.append(1)
        else:
            lb.append(0)
dfr['behind the scenes'] = lb            

            
for i in dfr['special_features']:
        if re.findall('Commentaries', i):
            lc.append(1)
        else:
            lc.append(0)
dfr['commentaries'] = lc
            
            
for i in dfr['special_features']:
        if re.findall('Deleted Scenes', i):
            ld.append(1)
        else:
            ld.append(0)
dfr['deleted scenes'] = lc

dfr = dfr.drop(['special_features'], axis=1)


In [271]:
dfr.duplicated().sum()

0

In [323]:
dfr

,film_id,rented_times,rental_duration,category,length,rating,test,trailers,behind the scenes,commentaries,deleted scenes
0,19,19,6,Action,113,PG,0,0,1,1,1
1,19,1,6,Action,113,PG,1,0,1,1,1
2,21,2,3,Action,129,R,1,0,1,1,1
3,21,19,3,Action,129,R,0,0,1,1,1
4,29,10,5,Action,168,NC-17,0,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...
1580,977,11,3,Travel,85,R,0,0,1,0,0
1581,981,21,7,Travel,55,NC-17,0,0,1,0,0
1582,988,15,7,Travel,139,R,0,1,1,1,1
1583,989,23,4,Travel,74,R,0,0,0,1,1


In [273]:
dfr = dfr[['film_id','genre','length','language','rating','rental_duration','rental_rate','test','trailers','behind the scenes','commentaries','deleted scenes']]


In [274]:
#dfr.info()
#dfr = dfr.drop_duplicates()


In [324]:
dfr['film_id'] = dfr['film_id'].astype(object)
dfr['rental_duration'] = dfr['rental_duration'].astype(object)
dfr['test'] = dfr['test'].astype(bool)
#dfr['rental_rate'] = dfr['rental_rate'].astype(object)
#dfr['trailers'] = dfr['trailers'].astype(bool)
#dfr['behind the scenes'] = dfr['behind the scenes'].astype(bool)
#dfr['commentaries'] = dfr['commentaries'].astype(bool)
#dfr['deleted scenes'] = dfr['deleted scenes'].astype(bool)


In [325]:
y = dfr['test']
X = dfr.drop('test', axis=1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1337)

In [326]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import StandardScaler

X_train_num = X_train.select_dtypes(include = np.number)

# Scaling data
transformer = MinMaxScaler().fit(X_train_num) # need to keep transformer
X_train_normalized = transformer.transform(X_train_num)
X_train_norm = pd.DataFrame(X_train_normalized)

In [327]:
X_train_norm.columns = X_train_num.columns
X_train_norm.head()

,rented_times,length,trailers,behind the scenes,commentaries,deleted scenes
0,0.862069,0.841727,1.0,1.0,0.0,0.0
1,0.689655,0.100719,0.0,1.0,1.0,1.0
2,0.689655,0.841727,1.0,1.0,1.0,1.0
3,0.241379,0.258993,1.0,1.0,0.0,0.0
4,0.000000,0.273381,0.0,1.0,1.0,1.0


In [328]:
X_train_cat = X_train.select_dtypes(include = np.object)



/var/folders/sm/8s2c1j894rq6v788yfhxt27r0000gn/T/ipykernel_5696/919404641.py:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_train_cat = X_train.select_dtypes(include = np.object)


In [329]:
X_train_cat

,film_id,rental_duration,category,rating
89,850,7,Action,R
1480,782,6,Sports,PG-13
876,500,5,Foreign,PG-13
1031,30,4,Horror,R
675,766,5,Drama,PG-13
...,...,...,...,...
1191,22,6,New,G
1256,686,5,New,R
860,431,3,Foreign,G
189,805,5,Animation,G


In [330]:
X_train_categorical = X_train.select_dtypes(include = np.object)
X_train_cat = pd.get_dummies(X_train_categorical, 
                             columns=['category', 'rating','rental_duration'],
                             drop_first=True)
X_train_cat.head()

/var/folders/sm/8s2c1j894rq6v788yfhxt27r0000gn/T/ipykernel_5696/1381724978.py:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_train_categorical = X_train.select_dtypes(include = np.object)


,film_id,category_Animation,category_Children,category_Classics,category_Comedy,category_Documentary,category_Drama,category_Family,category_Foreign,category_Games,...,category_Sports,category_Travel,rating_NC-17,rating_PG,rating_PG-13,rating_R,rental_duration_4,rental_duration_5,rental_duration_6,rental_duration_7
89,850,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
1480,782,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,1,0
876,500,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,1,0,0
1031,30,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
675,766,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,1,0,0


In [331]:
X_train_transformed = np.concatenate([X_train_norm, X_train_cat], axis=1)

In [332]:
from sklearn.linear_model import LogisticRegression
classification = LogisticRegression(random_state=0, solver='lbfgs',
                  multi_class='multinomial').fit(X_train_transformed, y_train)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [333]:
# for numericals
X_test_num = X_test.select_dtypes(include = np.number)

# Scaling data
# we use the transformer that was trained on the training data
X_test_normalized = transformer.transform(X_test_num)
X_test_norm = pd.DataFrame(X_test_normalized)

In [334]:
# for categoricals
X_test_categorical = X_test.select_dtypes(include = np.object)
X_test_cat = pd.get_dummies(X_test_categorical, 
                            columns=['category', 'rating','rental_duration'],
                            drop_first=True)
# verify that dummies columns are in the same order and that the same column was dropped
display(list(zip(list(X_train_cat.columns),list(X_test_cat.columns))))
# not needed if you treat each dataframe with one_hot_encoder and save the encode (and the column names)

X_test_cat.head()

/var/folders/sm/8s2c1j894rq6v788yfhxt27r0000gn/T/ipykernel_5696/1863834073.py:2: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_test_categorical = X_test.select_dtypes(include = np.object)


[('film_id', 'film_id'),
 ('category_Animation', 'category_Animation'),
 ('category_Children', 'category_Children'),
 ('category_Classics', 'category_Classics'),
 ('category_Comedy', 'category_Comedy'),
 ('category_Documentary', 'category_Documentary'),
 ('category_Drama', 'category_Drama'),
 ('category_Family', 'category_Family'),
 ('category_Foreign', 'category_Foreign'),
 ('category_Games', 'category_Games'),
 ('category_Horror', 'category_Horror'),
 ('category_Music', 'category_Music'),
 ('category_New', 'category_New'),
 ('category_Sci-Fi', 'category_Sci-Fi'),
 ('category_Sports', 'category_Sports'),
 ('category_Travel', 'category_Travel'),
 ('rating_NC-17', 'rating_NC-17'),
 ('rating_PG', 'rating_PG'),
 ('rating_PG-13', 'rating_PG-13'),
 ('rating_R', 'rating_R'),
 ('rental_duration_4', 'rental_duration_4'),
 ('rental_duration_5', 'rental_duration_5'),
 ('rental_duration_6', 'rental_duration_6'),
 ('rental_duration_7', 'rental_duration_7')]

,film_id,category_Animation,category_Children,category_Classics,category_Comedy,category_Documentary,category_Drama,category_Family,category_Foreign,category_Games,...,category_Sports,category_Travel,rating_NC-17,rating_PG,rating_PG-13,rating_R,rental_duration_4,rental_duration_5,rental_duration_6,rental_duration_7
1315,240,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
420,265,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
1070,665,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
1198,77,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
219,68,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0


In [335]:
X_test_transformed = np.concatenate([X_test_norm, X_test_cat], axis=1)

In [336]:
# Now we can make predictions on the test set:
predictions = classification.predict(X_test_transformed)
classification.score(X_test_transformed, y_test)

0.9716088328075709

In [337]:
print(y_test.value_counts())
# As you would notice here, there is a huge imbalance in the data among the different classes. 
# We will talk more about imbalance and how to resolve it later (tomorrow)


False    200
True     117
Name: test, dtype: int64


In [338]:
pd.Series(predictions).value_counts()
# This shows that the disparity in the numbers are amplified by the model

False    191
True     126
dtype: int64

In [339]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[191,   9],
       [  0, 117]])

In [340]:
# bonus: KNN classifier: look at nearest neighbours and use the majority to determine class
from sklearn import neighbors
clf = neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform')
clf.fit(X_train_transformed, y_train)
predictions_clf = clf.predict(X_test_transformed)
clf.score(X_test_transformed, y_test)

0.4952681388012618

In [347]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[191,   9],
       [  0, 117]])

In [ ]:
#rating - one hot encode
#DAYS rented. 
